In [1]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
from rail.raruma import utility_functions as raruma_util
from rail.raruma import plotting_functions as raruma_plot
from rail.raruma import admixture_functions as raruma_admix
from rail.raruma import wrapper_classes as raruma_wrap

In [2]:
# pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'  
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'
pz_dir = '/Users/echarles/pz'

In [3]:
train = tables_io.read(f"{pz_dir}/data/train/dp1_ecdfs_matched_specgold_train.hdf5")
test = tables_io.read(f"{pz_dir}/data/test/dp1_ecdfs_matched_specgold_test.hdf5")
#d.keys()
#train = tables_io.sliceObj(d, slice(0, -1, 10))
#test = tables_io.sliceObj(d, slice(1, -1, 10))
d = tables_io.read(f"{pz_dir}/data/on_sky/dp1_v29.0.0_gold_all.hdf5")
model_file = './model_inform_knn.pkl'

In [4]:
from rail.estimation.algos.k_nearneigh import KNearNeighEstimator
from rail.utils.catalog_utils import RubinCatalogConfig
RubinCatalogConfig.apply(RubinCatalogConfig.tag)
band_names = raruma_util.make_band_names('LSST_obs_{band}', 'ugrizy')

In [5]:
train_features = raruma_util.get_band_values(train, '{band}_gaap1p0Mag', 'ugrizy')

In [6]:
library = raruma_util.get_band_values(d, '{band}_gaap1p0Mag', 'ugrizy')

In [7]:
knn = KNearNeighEstimator.make_stage(name='knn', model=model_file, input='dummy.in', output_mode='return', nzbins=3001)
knn.stage_columns = knn.config.bands
knn_j = raruma_wrap.CatEstimatorWrapper(knn, band_names)

Inserting handle into data store.  model: ./model_inform_knn.pkl, knn


In [8]:
admix_grid = np.logspace(-4, 0, 17)

In [ ]:
the_dict = {}
knn_orig = knn_j(train_features.T)
means = []
stds = []
outlier_fracs = []
for admix in admix_grid:
    mixed_mags = raruma_admix.make_admixture(train_features, library, admixture=admix)
    knn_vals = knn_j(mixed_mags.T)
    deltas = knn_vals - knn_orig
    the_dict[admix] = deltas
    outliers = (np.fabs(deltas) > 0.15).sum() / float(deltas.size)
    outlier_fracs.append(outliers)
    means.append(deltas.mean())
    stds.append(deltas.std())

Process 0 estimating PZ PDF for rows 0 - 4,803
Inserting handle into data store.  output_knn: inprogress_output_knn.hdf5, knn
Process 0 estimating PZ PDF for rows 0 - 4,803
Inserting handle into data store.  output_knn: inprogress_output_knn.hdf5, knn
Process 0 estimating PZ PDF for rows 0 - 4,803
Inserting handle into data store.  output_knn: inprogress_output_knn.hdf5, knn
Process 0 estimating PZ PDF for rows 0 - 4,803


In [ ]:
_ = plt.plot(admix_grid, means, label=r"mean $\delta z$")
_ = plt.plot(admix_grid, stds, label=r"RMS $\delta z$")
_ = plt.plot(admix_grid, outlier_fracs, label=r"f $\delta z > 0.15$")
_ = plt.xscale('log')
_ = plt.legend()
_ = plt.xlabel("Flux Admixture Fraction")
_ = plt.ylabel(r"$\delta z$")

In [ ]:
for k, v in the_dict.items():
    _ = plt.hist(v, bins=np.linspace(-1, 1, 101), label=k)